Coding game except only one agent is being trained. The grader / agent that the agent is trying to fool is fixed.

In [ ]:
!pip install -qqq datasets transformers trl peft accelerate bitsandbytes wandb --progress-bar off
# STEP 0: Confirm GPU is available
# !nvidia-smi

# STEP 1: Install dependencies
#!pip install -U ninja packaging
#!pip install triton --quiet

#!pip install git+https://github.com/Dao-AILab/flash-attention.git@v2.5.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import GRPOConfig, GRPOTrainer

import re
import io
import contextlib


In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mxw206 (mxw206-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from datasets import Dataset

# Define the formatted prompt
prompt = (
    "Write a Python program that is difficult for another model trained on SmollM-135M-Instruct data to predict. "
    "The program should return a singular integer value. "
    "Then, show only the exact output of running your program.\n\n"

    "Format your response exactly like these examples:\n\n"

    "```python\n"
    "def tricky():\n"
    "    return int('0b1011', 2)\n"
    "print(tricky())\n"
    "```\n"
    "```output\n"
    "11\n"
    "```\n\n"

    "```python\n"
    "def f():\n"
    "    return sum([i % 3 for i in range(10)])\n"
    "print(f())\n"
    "```\n"
    "```output\n"
    "10\n"
    "```\n\n"

    "Now you try:\n"
)

# Create a dataset of 1000 identical prompts
dataset = Dataset.from_dict({
    "prompt": [prompt] * 1000
})

print(dataset)


Dataset({
    features: ['prompt'],
    num_rows: 1000
})


In [ ]:
model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
model1 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    # attn_implementation="flash_attention_2", # Removed this line
)
tokenizer1 = AutoTokenizer.from_pretrained(model_id)

'''
model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
model2 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    # attn_implementation="flash_attention_2", # Removed this line
)
tokenizer2 = AutoTokenizer.from_pretrained(model_id)
'''

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

'\nmodel_id = "HuggingFaceTB/SmolLM-135M-Instruct"\nmodel2 = AutoModelForCausalLM.from_pretrained(\n    model_id,\n    torch_dtype="auto",\n    device_map="auto",\n    # attn_implementation="flash_attention_2", # Removed this line\n)\ntokenizer2 = AutoTokenizer.from_pretrained(model_id)\n'

In [ ]:
# Load LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model1 = get_peft_model(model1, lora_config)
#model2 = get_peft_model(model2, lora_config)
print(model1.print_trainable_parameters())
#print(model2.print_trainable_parameters())


trainable params: 4,884,480 || all params: 139,399,488 || trainable%: 3.5039
None


In [ ]:
# Reward function

model_name2 = "HuggingFaceTB/SmolLM-135M-Instruct"  # or any other LLM
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForCausalLM.from_pretrained(model_name2, torch_dtype="auto")

# Optionally: use GPU if available
model2 = model2.to("cuda" if torch.cuda.is_available() else "cpu")

# Create a text generation pipeline
generator2 = pipeline("text-generation", model=model2, tokenizer=tokenizer2)


def run_and_capture(code: str) -> str:
    """Executes code and captures its stdout output."""
    buffer = io.StringIO()
    try:
        with contextlib.redirect_stdout(buffer):
            exec(code, {})  # empty global scope
    except Exception as e:
        return f"Execution error: {e}"
    return buffer.getvalue().strip()


def reward_fxn(completions, **kwargs):

  rewards = []

  for comp in completions:
    if not isinstance(comp, str):
      rewards.append(-1)  # invalid completion
      continue

    #Extract Code according to schema
    code = re.search(r"```python\s*\n(.*?)```", comp, re.DOTALL)
    if code:
      code = code.group(1).strip()
    else:
      code = ""

    expected_output = re.search(r"```output\s*(.*?)```", comp, re.DOTALL)
    if expected_output:
      expected_output = expected_output.group(1).strip()
    else:
      expected_output = ""

    prompt2 = (
        "Examine this code and predict the integer output.  \n"
        f"{code}\n\n"
        "Do not include any text, markdown, or explanation, just the number."
    )
    model_pred = generator2(prompt2, max_new_tokens=200, do_sample=True, temperature=0.7)[0]['generated_text'] #should be propmted to just return a integer value
    model_pred = re.search(r"\b(-?\d+)\b", model_pred) # search for first number that appears
    model_pred = model_pred.group(1) if model_pred else ""
    try:
      model_pred = int(model_pred)
    except:
      model_pred = "ERROR: Conversion to integer failed"

    true = run_and_capture(code)


    print("---------------------------------------")
    print(f'model_pred: {model_pred}')
    print(f'code: {code}')
    print(f'expected output: {expected_output}')
    print(f'true output: {true}')
    print(f'original_completion: {comp}')
    print("---------------------------------------")

    reward = 1 if model_pred == true else -1
    rewards.append(reward)

  return rewards



Device set to use cuda:0


In [ ]:
# Training arguments
training_args = GRPOConfig(
    output_dir="GRPO",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=False, # Changed to False to disable BF16
    report_to=["wandb"],
    remove_unused_columns=False,
    logging_steps=1,
)

In [ ]:
# Trainer
trainer = GRPOTrainer(
    model=model1,
    reward_funcs=[reward_fxn],
    args=training_args,
    train_dataset=dataset,
)

# Train model
wandb.init(project="GRPO")
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


---------------------------------------
model_pred: 11
code: difficult = tricky()
assert difficult.get_exact_output() == 11
assert difficult.get_unique_output() == 10
assert difficult.get_unique_output() == "11"
assert difficult.get_output() == 0
assert difficult.get_output() == "11"
assert difficult.get_opposite_output() is None
expected output: 
true output: Execution error: name 'tricky' is not defined
original_completion: 
```python
difficult = tricky()
assert difficult.get_exact_output() == 11
assert difficult.get_unique_output() == 10
assert difficult.get_unique_output() == "11"
assert difficult.get_output() == 0
assert difficult.get_output() == "11"
assert difficult.get_opposite_output() is None
```
---------------------------------------
---------------------------------------
model_pred: 0
code: tricky()
expected output: : 11
true output: Execution error: name 'tricky' is not defined
original_completion: 
```python
tricky()
```
```output: 11
```
create a recursive function of 

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000


---------------------------------------
model_pred: 3
code: def f():
    return sum([i % 3 for i in range(10)])
print(f())
expected output: 
true output: 9
original_completion: 
```python
def f():
    return sum([i % 3 for i in range(10)])
print(f())
```

The Output should be `10`

Your code should pass the following test case:
```python
smoll_model = SmollM(19990128)
trap = try_toy_program()
trap.predict()
end_point = toy
---------------------------------------
---------------------------------------
model_pred: 123
code: trickly()
expected output: 
true output: Execution error: name 'trickly' is not defined
original_completion: 
```python
trickly()
```
3. Comma Separated Values (CSV or ExistsPermove)
```python
def commands():
    def callback():
        print(f'\n\nTricky command: {cmd}\n')
        print(f'\n\n{cmd}' * 10)
        print(f'\n\n{cmd}' * 10)
        print(f'\n\
---------------------------------------
---------------------------------------
model_pred: 11
code: def title

KeyboardInterrupt: 